In [155]:
import csv

In [181]:
file_name = input("Train file name: " )

Train file name: input.txt


In [182]:
test_file_name = input("Test file name: " )

Test file name: test.txt


In [208]:
dist_calcu = input("Enter the dist calculating parameter: ")
if dist_calcu == "":
    dist_calcu = "e2"

Enter the dist calculating parameter: manh


In [216]:
k = input("Enter k value: ")
if k == "":
    k = int("3")
else:
    k = int(k)

Enter k value: 3


In [211]:
mes = input("Enter the weighting method: ")
if mes == "":
    mes = "1/d"

Enter the weighting method: unit


In [161]:
def read_file(file_name):
    if file_name[-3:] == "csv":
        lines = []
        with open('input.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for row in spamreader:
                lines.append(', '.join(row))
    elif file_name[-3:] == "txt":
        G = open(file_name,'r')
        with open(file_name) as G:
            lines = G.readlines()
        G.close()
    else:
        lines = "Unsupported file format"
        print("Unsupported file format")
    return lines

In [162]:
def getcom(l):
    spc = []
    for j in range(len(l)):
        if l[j] == ',':
            spc.append(j)
    return spc

In [163]:
def merge(lst):
    ret = []
    for l in lst:
        ret = ret + l
    
    return ret

In [164]:
def rm(var, dic):
    for v in dic:
        for ele in range(len(dic[v])):
            if dic[v][ele] == var:
                dic[v].remove(dic[v][ele])
                return dic

In [165]:
def getdist(p1, p2): #e2 distance
    d = 0
    for dim in range(len(p1)):
        d += (p2[dim]-p1[dim])**2
        
    return d

In [166]:
def manh(p1, p2):
    d = 0
    for dim in range(len(p1)):
        d += abs(p2[dim]-p1[dim])
    return d

In [167]:
def invert(dic):
    #print(dic)
    new = {}
    for v in dic:
        for e in dic[v]:
            if v not in new:
                new[v] = 1/max(e, 0.0001)
            else:
                new[v] += 1/max(e, 0.0001)
    #print(new)
    m_ = -1
    for v in new:
        if new[v] > m_:
            m_ = new[v]
    
    new = {value : key for (key, value) in new.items()}
    print(new[m_])
    return new[m_]

In [168]:
def unit(dic):
    ma = ""
    num = 0
    for v in dic:
        if len(dic[v]) > num:
            num = len(dic[v])
    
    for v in dic:
        if len(dic[v]) == num:
            print(v)
            return v

In [169]:
def getpoints(lines):
    points = {}

    for l in lines:
        temp = l.replace("\n", "")

        com = getcom(temp)
        com.insert(0, -1)
        com.append(len(temp))

        temp_p = []

        for pos in range(1, len(com)):
            if temp[com[pos-1]+1:com[pos]].isdigit():
                temp_p.append(int(temp[com[pos-1]+1:com[pos]]))
            else:
                temp_p.append(temp[com[pos-1]+1:com[pos]])

        #print(temp_p)

        if temp_p[-1] not in points:
            points[temp_p[-1]] = [temp_p[:-1]]
        else:
            points[temp_p[-1]].append(temp_p[:-1])

    #print(points)
    return points

In [179]:
def prec_recall(pred, test_points, points):
    track = []
    for cl in test_points:
        TP = 0
        for pt in test_points[cl]:
            if pt in pred[cl]:
                TP += 1
        track.append(cl)
        print("Label = " + str(cl) + " Precision = " + str(TP) + "/" + str(len(pred[cl])) + " Recall = " + str(TP) + "/" + str(len(test_points[cl])))
    
    for cl in points:
        if cl not in track:
            a = 0
            b = 0
            if cl in pred:
                a = len(pred[cl])
            if cl in test_points:
                b = len(test_points[cl])
            print("Label = " + str(cl) + " Precision = " + str(0) + "/" + str(a) + " Recall = " + str(0) + "/" + str(b))

In [171]:
def gettest(lines):
    import copy

    A = copy.deepcopy(lines)
    lines = []
    for l in A:
        temp = l.replace("\n", "")
        comi = getcom(temp)
        lines.append(temp[:comi[-1]])

    #print(lines)
    
    test = []

    for l in lines:
        temp = l.replace("\n", "")

        com = getcom(temp)
        com.insert(0, -1)
        com.append(len(temp))

        temp_p = []

        for pos in range(1, len(com)):
            if temp[com[pos-1]+1:com[pos]].isdigit():
                temp_p.append(int(temp[com[pos-1]+1:com[pos]]))
            else:
                temp_p.append(temp[com[pos-1]+1:com[pos]])

        test.append(temp_p)

    #print(test)
    return test

In [172]:
def mainrun(test, points, dist_calcu, k, mes):
    pred = {}
    for i in range(len(test)):
        dist = {}

        for types in points:
            for p in points[types]:
                if dist_calcu == "e2":
                    d = getdist(p, test[i])
                else:
                    d = manh(p, test[i])
                if types not in dist:
                    dist[types] = [d]
                else:
                    dist[types].append(d)

        #print(dist)
        min_dist = {}

        for types in dist:
            for d in dist[types]:
                ret = merge(list(min_dist.values()))
                if types not in min_dist and len(ret) < k:
                    min_dist[types] = [d]
                else:
                    if len(ret) < k:
                        min_dist[types].append(d)
                    else:
                        m = max(ret)
                        if d < m:
                            min_dist = rm(m, min_dist)
                            if types not in min_dist:
                                min_dist[types] = [d]
                            else:
                                min_dist[types].append(d)


        #print(test[i])
        #print(min_dist)
        print(test[i])
        if len(min_dist) == 1:
            print(list(min_dist.keys())[0])
            if list(min_dist.keys())[0] in pred:
                pred[list(min_dist.keys())[0]].append(test[i])
            else:
                pred[list(min_dist.keys())[0]] = [test[i]]
        else:
            if mes == "1/d":
                ret = invert(min_dist)
                if ret in pred:
                    pred[ret].append(test[i])
                else:
                    pred[ret] = [test[i]]
            elif mes == "unit":
                ret = unit(min_dist)
                if ret in pred:
                    pred[ret].append(test[i])
                else:
                    pred[ret] = [test[i]]
        print("\n")
    return pred

In [212]:
lines = read_file(file_name)
points = getpoints(lines)
test_lines = read_file(test_file_name)
test_points = getpoints(test_lines)
test = gettest(test_lines)
pred = mainrun(test, points, dist_calcu, k, mes)
prec_recall(pred, test_points, points)

[3, 3, 6]
A


[4, 5, 20]
A


[2, 2, 33]
C


[10, 9, 19]
A


[14, 12, 100]
B


[17, 19, 101]
B


[65, 82, 29]
C


[48, 39, 37]
C


[70, 48, 33]
C


[38, 75, 7]
C


[79, 51, 94]
B


[93, 41, 51]
B


[90, 14, 18]
C


[52, 23, 62]
B


[78, 79, 33]
C


[90, 96, 10]
C


[90, 9, 42]
C


[81, 20, 46]
C


[73, 68, 61]
B


[79, 99, 85]
B


[23, 5, 58]
C


[71, 59, 1]
C


[78, 8, 26]
C


[78, 72, 62]
B


[10, 64, 76]
B


[16, 61, 12]
C


[42, 1, 5]
A


[31, 41, 35]
C


[36, 49, 96]
B


[75, 29, 74]
B


[95, 96, 0]
C


[23, 47, 74]
B


[91, 20, 86]
B


[50, 54, 87]
B


[14, 1, 35]
C


[87, 15, 52]
C


[24, 53, 26]
C


[36, 15, 5]
A


[26, 50, 17]
C


[85, 3, 2]
C


[52, 89, 85]
B


[4, 94, 46]
C


[37, 55, 51]
B


[19, 40, 54]
C


[83, 43, 87]
B


[44, 3, 15]
A


[33, 63, 21]
C


[49, 97, 69]
B


[11, 49, 49]
C


[66, 65, 26]
C


[31, 84, 94]
B


[23, 42, 47]
C


[76, 0, 71]
B


[92, 36, 49]
C


[26, 98, 64]
B


[55, 23, 41]
C


[69, 68, 17]
C


[25, 5, 14]
A


[92, 59, 37]
C


[26, 43, 33]
C


[4